In [83]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [84]:
sns.set_palette("colorblind")

In [85]:
cart = pd.read_csv("dataset-carte.csv")
df = pd.read_csv("dataset-entier.csv")
cont = pd.read_csv("liste-contaminants.csv")

In [86]:
df.head()


,bss_id,au_moins_un_depassement,au_moins_un_20_pourcent_depassement,params_recherches,params_depassement,mesures_depassement,dates_depassement,count_params_depassement,pourcentage_depassement,total_mesures,...,mesures_quantif,dates_quantif,count_params_quantif,nb_quantif_per_param,params_potabilite,mesures_potabilite,dates_potabilite,count_params_potabilite,alerte_nitrates,famille
0,BSS001MWMQ,False,False,1335|1339|1340,NaN,NaN,NaN,0,NaN,NaN,...,"[30.6, 29.9, 31.0, 32.0]","['2016-11-07', '2018-05-23', '2020-10-06', '20...",1,4,NaN,NaN,NaN,0,4,nitrates et nitrites
1,BSS001MWNL,True,True,1433|1335|1339|1340,1433,"[2.0, 2.22, 1.75, 1.69, 1.87, 1.67, 1.72, 1.69...","['2016-10-10', '2017-05-10', '2017-09-19', '20...",1,100.0,13.0,...,"[0.012, 0.009, 0.023, 0.007, 0.015, 0.007, 0.0...","['2017-05-10', '2017-09-19', '2018-04-10', '20...",3,7|2|13,NaN,NaN,NaN,0,11,nitrates et nitrites
2,BSS001MWNV,False,False,1433|1335|1339|1340,NaN,NaN,NaN,0,NaN,NaN,...,"[0.36, 0.36, 0.338, 0.332, 0.37, 0.326, 0.354,...","['2016-04-05', '2016-10-10', '2017-05-10', '20...",3,15|8|20,NaN,NaN,NaN,0,0,nitrates et nitrites
3,BSS001MWNW,False,False,1433|1335|1339|1340,NaN,NaN,NaN,0,NaN,NaN,...,"[0.19, 0.21, 0.173, 0.175, 0.19, 0.231, 0.266,...","['2016-04-05', '2016-10-10', '2017-05-10', '20...",3,15|8|15,NaN,NaN,NaN,0,0,nitrates et nitrites
4,BSS001MWRU,False,False,1335|1339|1340,NaN,NaN,NaN,0,NaN,NaN,...,"[3.4, 3.7, 4.1, 3.4]","['2016-04-11', '2018-10-17', '2020-02-17', '20...",1,4,NaN,NaN,NaN,0,0,nitrates et nitrites


In [87]:
df.iloc[10304]

bss_id                                                                        BSS001MWMQ
au_moins_un_depassement                                                            False
au_moins_un_20_pourcent_depassement                                                False
params_recherches                      1386|1388|1337|1338|1362|1369|1375|1376|1385|7073
params_depassement                                                                   NaN
mesures_depassement                                                                  NaN
dates_depassement                                                                    NaN
count_params_depassement                                                               0
pourcentage_depassement                                                              NaN
total_mesures                                                                        NaN
params_quantif                                        1337|1338|1362|1369|1375|1376|7073
mesures_quantif      

In [88]:
df["params_depassement"].nunique()

1351

In [89]:
cart.dtypes

Unnamed: 0                                 int64
bss_id                                    object
params_depassement                        object
max_mesure_per_param_depassement          object
nb_depassement_per_param                  object
max_mesure_per_param_depassement_date     object
params_quantif                            object
nb_quantif_per_param                      object
famille                                   object
x                                        float64
y                                        float64
nom_commune                               object
num_departement                          float64
alerte_nitrates                            int64
count_params_potabilite                    int64
codes_masse_eau_edl                       object
count_params_depassement                   int64
dtype: object

In [105]:
# Ensure the 'params_depassement' values are strings
cart['params_depassement'] = cart['params_depassement'].astype(str)
cont['code_sandre'] = cont['code_sandre'].astype(str)

# Split the 'params_depassement' column into separate columns
split_params = cart['params_depassement'].str.split('|', expand=True)

# Rename the split columns (optional)
split_params.columns = [f'param_{i+1}' for i in split_params.columns]

# Combine the 'x', 'y', and split columns
result = pd.concat([cart[['x', 'y', 'nom_commune','num_departement']], split_params], axis=1)

# Create a mapping dictionary from cont DataFrame
mapping_dict = pd.Series(cont.nom.values, index=cont.code_sandre).to_dict()

# Apply the mapping to each split column in the result DataFrame
for col in split_params.columns:
    result[col] = result[col].map(mapping_dict)

print(result)

           x       y                 nom_commune  num_departement  \
0     -2.955  48.617                    Pléguien             22.0   
1      5.574  47.327                   Chevigney             70.0   
2      5.592  45.458  Saint-Sulpice-des-Rivoires             38.0   
3      5.666  45.506       Saint-Jean-d'Avelanne             38.0   
4      6.710  45.503        La Plagne Tarentaise             73.0   
...      ...     ...                         ...              ...   
24690  0.700  49.695           Héricourt-en-Caux             76.0   
24691  2.313  48.947       Villeneuve-la-Garenne             92.0   
24692  0.932  49.585                      Limésy             76.0   
24693  5.757  47.390                       Choye             70.0   
24694  5.026  49.260                      Baulny             55.0   

             param_1       param_2              param_3      param_4  \
0                NaN           NaN                  NaN          NaN   
1                NaN       

In [106]:
result.nunique()

x                   9090
y                   7773
nom_commune        11860
num_departement       98
param_1               85
param_2              109
param_3              109
param_4               93
param_5               74
param_6               64
param_7               55
param_8               52
param_9               39
param_10              32
param_11              31
param_12              31
param_13              24
param_14              23
param_15              21
param_16              17
param_17              15
param_18              14
param_19               9
param_20              11
param_21               9
param_22               9
param_23               7
param_24               7
param_25               7
param_26               5
param_27               4
param_28               4
param_29               3
param_30               3
param_31               3
param_32               1
param_33               1
param_34               0
param_35               0
dtype: int64

In [107]:
cart['params_depassement'] = cart['params_depassement'].astype(str)
cont['code_sandre'] = cont['code_sandre'].astype(str)

# Split the 'params_depassement' column into separate columns
split_params = cart['params_depassement'].str.split('|', expand=True)

# Rename the split columns (optional)
split_params.columns = [f'param_{i+1}' for i in split_params.columns]

# Combine the 'x', 'y', and split columns
result = pd.concat([cart[['x', 'y', 'nom_commune','num_departement' ]], split_params], axis=1)

# Create a mapping dictionary from cont DataFrame
mapping_dict = pd.Series(cont.nom.values, index=cont.code_sandre).to_dict()
usage_lm_dict = pd.Series(cont.usage_lm.values, index=cont.nom).to_dict()

# Apply the mapping to each split column in the result DataFrame
for col in split_params.columns:
    result[col] = result[col].map(mapping_dict)

# List of categories
categories = ['pesticides et leurs métabolites',
              'métaux, métalloïdes et autres ions minéraux',
              'hydrocarbures aromatiques polycycliques',
              'substances chimiques industrielles (utilisation et/ou production)',
              'médicaments', 'nitrates, nitrites et engrais azotés']

# Initialize the category count columns with zero
for category in categories:
    result[category] = 0

# Count the occurrences of each category for each row
for index, row in result.iterrows():
    for col in split_params.columns:
        if pd.notna(row[col]):
            category = usage_lm_dict.get(row[col], None)
            if category in categories:
                result.at[index, category] += 1

print(result)

           x       y                 nom_commune  num_departement  \
0     -2.955  48.617                    Pléguien             22.0   
1      5.574  47.327                   Chevigney             70.0   
2      5.592  45.458  Saint-Sulpice-des-Rivoires             38.0   
3      5.666  45.506       Saint-Jean-d'Avelanne             38.0   
4      6.710  45.503        La Plagne Tarentaise             73.0   
...      ...     ...                         ...              ...   
24690  0.700  49.695           Héricourt-en-Caux             76.0   
24691  2.313  48.947       Villeneuve-la-Garenne             92.0   
24692  0.932  49.585                      Limésy             76.0   
24693  5.757  47.390                       Choye             70.0   
24694  5.026  49.260                      Baulny             55.0   

             param_1       param_2              param_3      param_4  \
0                NaN           NaN                  NaN          NaN   
1                NaN       

In [108]:
result.nunique()

x                                                                     9090
y                                                                     7773
nom_commune                                                          11860
num_departement                                                         98
param_1                                                                 85
param_2                                                                109
param_3                                                                109
param_4                                                                 93
param_5                                                                 74
param_6                                                                 64
param_7                                                                 55
param_8                                                                 52
param_9                                                                 39
param_10                 

In [110]:
cart.head()

,Unnamed: 0,bss_id,params_depassement,max_mesure_per_param_depassement,nb_depassement_per_param,max_mesure_per_param_depassement_date,params_quantif,nb_quantif_per_param,famille,x,y,nom_commune,num_departement,alerte_nitrates,count_params_potabilite,codes_masse_eau_edl,count_params_depassement
0,1,BSS000QXYD,nan,NaN,NaN,NaN,;;;;;,;;;;;,;;;;4;,-2.955,48.617,Pléguien,22.0,0,0,GG039,0
1,2,BSS001JDCY,nan,NaN,NaN,NaN,1107|6276;;;;;1386|1337|1338|1362|1369|1375|7073,1|1;;;;;1|2|2|2|2|2|2,0;1;2;;4;5,5.574,47.327,Chevigney,70.0,0,0,NaN,0
2,3,BSS001VUGC,nan,NaN,NaN,NaN,1108|6276|6854;;2013;;1340;1337|1338|1375|7073,3|4|1;;1;;8;8|8|8|2,0;1;2;;4;5,5.592,45.458,Saint-Sulpice-des-Rivoires,38.0,0,0,DG350,0
3,4,BSS001VUGK,nan,NaN,NaN,NaN,1263|1108|6276;;;;1340;1337|1338|1362|1375|7073,1|1|1;;;;1;1|1|1|1|1,0;1;2;;4;5,5.666,45.506,Saint-Jean-d'Avelanne,38.0,0,0,NaN,0
4,5,BSS001VXSN,nan,NaN,NaN,NaN,;;;;1433|1340;1337|1338|1369|1370|1375|1396|7073,;;;;5|5;1|5|3|1|5|5|4,0;1;2;;4;5,6.710,45.503,La Plagne Tarentaise,73.0,0,0,DG406,0


In [95]:
cont["usage_lm"].unique()

array(['pesticides et leurs métabolites',
       'métaux, métalloïdes et autres ions minéraux',
       'hydrocarbures aromatiques polycycliques ',
       'substances chimiques industrielles (utilisation et/ou production)',
       'médicaments', 'nitrates, nitrites et engrais azotés'],
      dtype=object)

In [96]:
cont.loc[cont["code_sandre"] == 1386 ]

,code_sandre,nom,code_cas,usage_lm,metabolite_lm,unite_ades,valeur_seuil_lm,unite_valeur_limite_conso_humaine,valeur_limite_conso_humaine,commentaire_lm


cont.loc[cont["usage_lm"] == "métaux, métalloïdes et autres ions minéraux"]

In [97]:
cont.loc[cont["usage_lm"] == "métaux, métalloïdes et autres ions minéraux"]

,code_sandre,nom,code_cas,usage_lm,metabolite_lm,unite_ades,valeur_seuil_lm,unite_valeur_limite_conso_humaine,valeur_limite_conso_humaine,commentaire_lm
159,1382,plomb,7439-92-1,"métaux, métalloïdes et autres ions minéraux",non applicable,µg/L,10,µg/L,50,NaN
160,1386,nickel,7440-02-0,"métaux, métalloïdes et autres ions minéraux",non applicable,µg/L,20,µg/L,20,NaN
161,1387,mercure,7439-97-6,"métaux, métalloïdes et autres ions minéraux",non applicable,µg/L,1,µg/L,1,NaN
162,1388,cadmium,7440-43-9,"métaux, métalloïdes et autres ions minéraux",non applicable,µg/L,5,µg/L,5,NaN
163,1337,chlorures,16887-00-6,"métaux, métalloïdes et autres ions minéraux",non applicable,mg/L,250,mg/L,200,limite conso humaine inférieure au seuil 2023
164,1338,sulfate,14808-79-8,"métaux, métalloïdes et autres ions minéraux",non applicable,mg/L,250,mg/L,250,"naturellement il y a des évaporites, difficile..."
165,1362,bore,7440-42-8,"métaux, métalloïdes et autres ions minéraux",non applicable,µg/L,1000,µg/L,2400,"La limite de qualité est fixée à 2,4 mg/ L lor..."
166,1369,arsenic,7440-38-2,"métaux, métalloïdes et autres ions minéraux",non applicable,µg/L,10,µg/L,100,NaN
167,1370,aluminium,7429-90-5,"métaux, métalloïdes et autres ions minéraux",non applicable,µg/L,200,NaN,NaN,NaN
168,1371,chrome hexavalent,18540-29-9,"métaux, métalloïdes et autres ions minéraux",non applicable,µg/L,50,µg/L,50,NaN


In [98]:
result.nunique()

x                                                                    9090
y                                                                    7773
param_1                                                                85
param_2                                                               109
param_3                                                               109
param_4                                                                93
param_5                                                                74
param_6                                                                64
param_7                                                                55
param_8                                                                52
param_9                                                                39
param_10                                                               32
param_11                                                               31
param_12                              

In [99]:
# Correctly creating mapping dictionary
mapping_dict = pd.Series(cont['nom'].values, index=cont['code_sandre']).to_dict()
usage_lm_dict = pd.Series(cont['usage_lm'].values, index=cont['nom']).to_dict()

# Display the dictionaries to verify
print("\nMapping from code_sandre to nom:")
print(mapping_dict)
print("\nMapping from nom to usage_lm:")
print(usage_lm_dict)



Mapping from code_sandre to nom:
{'1101': 'alachlore', '1103': 'aldrine', '1105': 'aminotriazole', '1113': 'bentazone', '1129': 'carbendazime', '1136': 'chlortoluron', '1139': 'cymoxanil', '1173': 'dieldrine', '1177': 'diuron', '1185': 'fénarimol', '1197': 'heptachlore', '1206': 'iprodione', '1208': 'isoproturon', '1209': 'linuron', '1228': 'monuron', '1257': 'propiconazole', '1263': 'simazine', '1280': 'triadiménol', '1403': 'diméthomorphe', '1414': 'propyzamide', '1473': 'chlorothalonil', '1474': 'chlorprophame', '1479': 'dibromo-1,2 chloro-3 propane', '1498': 'dibromoéthane-1,2', '1528': 'pirimicarbe', '1655': 'dichloropropane-1,2', '1662': 'sulcotrione', '1688': 'aclonifène', '1694': 'tébuconazole', '1704': 'imazalil', '1744': 'epoxiconazole', '1748': 'heptachlore époxyde exo cis', '1796': 'métaldéhyde', '1866': 'chlordécone', '1879': 'metconazole', '1903': 'acétochlore', '2076': 'mésotrione', '2986': 'imazamox', '6390': 'thiamethoxam', '1083': 'chlorpyriphos-éthyl', '1092': 'pros

In [100]:
# Flatten the data to create a list of all parameters
all_params = result.filter(like='param_').values.flatten()
# Remove None and NaN from the list to ensure clean data for counting
all_params = all_params[pd.notna(all_params)]

# Print some of the flattened data to verify
print("Sample of flattened data:", all_params[:10])

# Count occurrences of each unique parameter
param_counts = pd.Series(all_params).value_counts().reset_index()
param_counts.columns = ['nom', 'count']

# Print counts to verify
print("\nCounts of unique parameters:")
print(param_counts.head())

# Apply the usage_lm mapping
param_counts['usage_lm'] = param_counts['nom'].map(usage_lm_dict)

# Print DataFrame after applying the mapping
print("\nDataFrame after applying 'usage_lm' mapping:")
print(param_counts.head())

# Clean up DataFrame by dropping any rows with NaN in 'nom'
param_counts.dropna(subset=['nom'], inplace=True)

# Final DataFrame to display
print("\nFinal DataFrame with counts and category mappings:")
print(param_counts)

# Check if DataFrame is still empty
if param_counts.empty:
    print("No data to display after processing.")
else:
    print("Data processing successful.")




Sample of flattened data: ['fluorure' 'glyphosate' 'atrazine déséthyl' 'glyphosate' 'atrazine'
 'somme des pesticides' 'arsenic' 'triclopyr'
 'atrazine déisopropyl déséthyl' 'desphenyl-chloridazon']

Counts of unique parameters:
                             nom  count
0           somme des pesticides   1455
1          desphenyl-chloridazon   1157
2              atrazine déséthyl    825
3  atrazine déisopropyl déséthyl    784
4                      bentazone    243

DataFrame after applying 'usage_lm' mapping:
                             nom  count                         usage_lm
0           somme des pesticides   1455  pesticides et leurs métabolites
1          desphenyl-chloridazon   1157  pesticides et leurs métabolites
2              atrazine déséthyl    825  pesticides et leurs métabolites
3  atrazine déisopropyl déséthyl    784  pesticides et leurs métabolites
4                      bentazone    243  pesticides et leurs métabolites

Final DataFrame with counts and category mappi

In [101]:
param_counts

,nom,count,usage_lm
0,somme des pesticides,1455,pesticides et leurs métabolites
1,desphenyl-chloridazon,1157,pesticides et leurs métabolites
2,atrazine déséthyl,825,pesticides et leurs métabolites
3,atrazine déisopropyl déséthyl,784,pesticides et leurs métabolites
4,bentazone,243,pesticides et leurs métabolites
...,...,...,...
165,heptachlore,1,pesticides et leurs métabolites
166,molybdène,1,"métaux, métalloïdes et autres ions minéraux"
167,heptachlore époxyde endo trans,1,pesticides et leurs métabolites
168,fipronil,1,pesticides et leurs métabolites


In [111]:
param_counts.to_csv("param_counts.csv")

In [112]:
result.to_csv("map_with_family.csv")